# Geo part

In [35]:
import json, logging
from geopy.geocoders import Nominatim

def seperator(a,b):
    c = (b - a)/10
    return [round(a,7),round(a+c,7),round(a+c*2,7),round(a+c*3,7),round(a+c*4,7),round(a+c*5,7),round(a+c*6,7),round(a+c*7,7),round(a+c*8,7),round(b,7)]

def geo_point_creator(city):
    geolocator = Nominatim(user_agent="ny_explorer")
    location = geolocator.geocode(city)
    lat_list = seperator(float(location.raw['boundingbox'][0]),float(location.raw['boundingbox'][1]))
    lon_list = seperator(float(location.raw['boundingbox'][2]),float(location.raw['boundingbox'][3]))
    point_list = []
    for lat in lat_list:
        for lon in lon_list:
            point_list.append([lat,lon])
    city_points[city] = point_list

# Data Cleaning

In [130]:
import pandas as pd
import numpy as np
import time
import requests

df = pd.read_csv('CITIES_12062020101058739.csv')

In [131]:
gdp_2018 = df[df['Year']==2018]
top5_gdpt_city = gdp_2018[gdp_2018['VAR']=='GDP_REAL_PPP'].sort_values(by='Value',ascending=False).head()
gdpt = top5_gdpt_city['Metropolitan areas'].to_list()

In [132]:
top5_gdpp_city = gdp_2018[gdp_2018['VAR']=='GDP_PC_REAL_PPP'].sort_values(by='Value',ascending=False).head()
gdpp = top5_gdpp_city['Metropolitan areas'].to_list()

In [6]:
CLIENT_ID = 'GBHOGHOE3DL1WYFOB4W5WI5SM1PLH1MDY4WN3O12SFGOVDUU' # your Foursquare ID
CLIENT_SECRET = 'AI25GODMBYEC331GD1VSY4Y3MRBP3SKR15N2YC01VKLSR0U4' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)
LIMIT = 100

Your credentails:
CLIENT_ID: GBHOGHOE3DL1WYFOB4W5WI5SM1PLH1MDY4WN3O12SFGOVDUU
CLIENT_SECRET:AI25GODMBYEC331GD1VSY4Y3MRBP3SKR15N2YC01VKLSR0U4


In [126]:
def getNearbyVenues(name, latitudes, longitudes, radius=5000):
    
    venues_list=[]
    for lat, lng in zip(latitudes, longitudes):
        print('{},{}'.format(lat,lng))
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name,
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['City Name',
                  'Latitude', 
                  'Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

# Test Zone

In [127]:
test_venues = getNearbyVenues(name='New York',latitudes=df2['lat'],longitudes=df2['lng'])

40.477399,-74.25909
40.477399,-74.2031991
40.477399,-74.1473082
40.477399,-74.0914173
40.477399,-74.0355264


In [91]:
df = pd.DataFrame(city_points['New York'],columns=['lat','lng'])

In [93]:
df2 = df.head()

,lat,lng
0,40.477399,-74.259090
1,40.477399,-74.203199
2,40.477399,-74.147308
3,40.477399,-74.091417
4,40.477399,-74.035526
...,...,...
95,40.916179,-73.979636
96,40.916179,-73.923744
97,40.916179,-73.867854
98,40.916179,-73.811963


In [128]:
test_venues

,City Name,Latitude,Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,New York,40.477399,-74.259090,Raritan Bay Waterfront Park,40.476497,-74.269338,Park
1,New York,40.477399,-74.259090,Fernandes Steakhouse 3,40.465149,-74.264378,Brazilian Restaurant
2,New York,40.477399,-74.259090,South Amboy YMCA,40.477545,-74.275680,Gym / Fitness Center
3,New York,40.477399,-74.259090,Sciortino's Harbor Lights,40.481881,-74.279224,Pizza Place
4,New York,40.477399,-74.259090,Blue Moon,40.482594,-74.279607,Bar
...,...,...,...,...,...,...,...
199,New York,40.477399,-74.035526,Observation Deck,40.471115,-74.005559,Scenic Lookout
200,New York,40.477399,-74.035526,Seastreak - Sandy Hook Dock,40.464688,-74.006465,Pier
201,New York,40.477399,-74.035526,Sandy Hook Nude Beach,40.459837,-73.988757,Nudist Beach
202,New York,40.477399,-74.035526,Sandy Hook Bay Side,40.448619,-73.999310,Beach


In [71]:
lat = 53.567411	
lng = -2.474144
radius = 500
LIMIT = 100
    
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)

In [72]:
results = requests.get(url).json()["response"]['groups'][0]['items']

In [56]:
url

'https://api.foursquare.com/v2/venues/explore?&client_id=GBHOGHOE3DL1WYFOB4W5WI5SM1PLH1MDY4WN3O12SFGOVDUU&client_secret=AI25GODMBYEC331GD1VSY4Y3MRBP3SKR15N2YC01VKLSR0U4&v=20180605&ll=53.567411,-2.474144&radius=500&limit=100'

In [87]:
city_points = {}
geo_point_creator('New York')

In [89]:
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode('New York')

In [90]:
print(json.dumps(location.raw,indent=4,ensure_ascii=False))

{
    "place_id": 234930245,
    "licence": "Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright",
    "osm_type": "relation",
    "osm_id": 175905,
    "boundingbox": [
        "40.477399",
        "40.9161785",
        "-74.25909",
        "-73.7001809"
    ],
    "lat": "40.7127281",
    "lon": "-74.0060152",
    "display_name": "New York, United States of America",
    "class": "boundary",
    "type": "administrative",
    "importance": 1.017576611451846,
    "icon": "https://nominatim.openstreetmap.org/images/mapicons/poi_boundary_administrative.p.20.png"
}
